In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import gc

from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf
%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [3]:
   
kfold = StratifiedKFold(n_splits=20, shuffle=True)
cvscores = []

for train, test in kfold.split(x_train, y_train):
    num_classes = 2
    high = 80
    
    filter_lenghts =  [int(i) for i in np.arange(2,high,2)]
    print(filter_lenghts)
    convs = []
    maxlen = 298
    batch_size = 1024
    epochs = 30
    numFilters=128
    activations= "sigmoid"
    dropoutVal = 0.2485206320388983
    lr = 0.001362881122433337
    adam = keras.optimizers.Adam(lr=lr)


    optim = adam
    y_train_k = keras.utils.to_categorical(y_train[train], num_classes)
    y_test_k = keras.utils.to_categorical(y_train[test], num_classes)
    
    input_flow = Input(shape=input_shape)

    convs= {}
    mpoolings = {}
    flattens = {}
    convs_out = []
    for i in filter_lenghts:
        convs[str(i)+'_convolution']=Conv1D(filters=numFilters,kernel_size=i,padding="valid",activation=activations,strides=1)(input_flow)

        mpoolings[str(i)+'_maxpooling'] = MaxPooling1D(pool_size= maxlen - i + 1)(convs[str(i)+'_convolution'])
        flattens[str(i)+'_flattenout'] = Flatten()(mpoolings[str(i)+'_maxpooling'])
        convs_out.append(flattens[str(i)+'_flattenout'])
    out = Concatenate()(convs_out)
    dropout = Dropout(dropoutVal)(out)
    dense = Dense(64, activation='relu')(dropout)
    dense2 = Dense(32, activation='relu')(dense)
    dropout2 = Dropout(dropoutVal)(dense2)
    end = Dense(num_classes, activation='softmax')(dropout2)

    model = Model(inputs=input_flow, outputs=end) 
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.fit(x_train[train],y_train_k, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_train[test], y_test_k, verbose=0)
    
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    del model
    gc.collect()
    K.clear_session()
    
print("%.5f%% (+/- %.5f%%)" % (np.mean(cvscores), np.std(cvscores)))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 88.57413%
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]
accuracy: 87.48820%
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]
accuracy: 87.58262%
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]
accuracy: 88.29084%
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78]
accuracy: 88.47970%
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22

In [4]:
cvscores

[88.57412934303284,
 87.48819828033447,
 87.58262395858765,
 88.29084038734436,
 88.47969770431519,
 88.14919590950012,
 87.77148127555847,
 88.62134218215942,
 85.78848242759705,
 89.14070129394531,
 88.19640874862671,
 89.28233981132507,
 87.58262395858765,
 88.81019949913025,
 88.05477023124695,
 87.44097948074341,
 87.9603385925293,
 88.04912567138672,
 88.04912567138672,
 89.13556933403015]